#### Multiclass classification

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from torch import nn

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
from helper_functions import plot_predictions, plot_decision_boundary

In [ ]:
# set hyperparameters
NUM_CLASSES = 4
NUM_FEATURES = 2
RANDOM_SEED = 42

# 1. create multiclass data
X_blob, y_blob = make_blobs(n_samples=1000, n_features=NUM_FEATURES, centers=NUM_CLASSES, cluster_std=1.5, random_state=RANDOM_SEED)

# 2. turn data into tensors
X_blob = torch.from_numpy(X_blob).type(torch.LongTensor).to(device=device)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor).to(device=device)

# 3. split into train and test
X_blob_train, X_blob_test, y_blob_train, y_blob_test = train_test_split(X_blob, y_blob, test_size=0.3, random_state=RANDOM_SEED)

# 4. plot data
plt.figure(figsize=(10, 7))
plt.scatter(X_blob[:, 0], X_blob[:, 1], c=y_blob, cmap=plt.cm.RdYlBu)

#### Model

In [ ]:

class BlobModel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
        """ initializes multiclass classification.
        
        Args:
            input_features (int): Number of input features to the model
            output_features (int): Number of output features (number of output class)
            hidden_units (int): Number of hidden units between layers, default 8

        Returns:
        
        Examples:
        """
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            # nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            # nn.ReLu(),
            nn.Linear(in_features=hidden_units, out_features=output_features)
        )

    def forward(self, x):
        return self.linear_layer_stack(x)


In [ ]:
torch.unique(y_blob_train)

In [ ]:
model_4 = BlobModel(input_features=2, output_features=4).to(device)

In [ ]:
# create loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_4.parameters(), lr=0.1)


### Getting prediction probabilities for multiclass classification
In order to evaluate and train and test our model, we need to convert our outputs (logits) to prediction probabilities and then to prediction labels.

logits (raw output) -> prediction probabilities (use softmax) -> prediction labels (use argmax of the prediction probabilities)

In [ ]:

X_blob_train = X_blob_train.float()
X_blob_test = X_blob_test.float()

model_4.eval()
with torch.inference_mode():
    y_logits = model_4(X_blob_test.to(device))
    y_pred_probs = torch.softmax(y_logits, dim=1)

print(y_logits[:5])
print(y_pred_probs[:5])

In [ ]:
y_pred_probs[0]

In [ ]:
torch.sum(y_pred_probs[0])

In [ ]:
torch.max(y_pred_probs[0])

In [ ]:
torch.argmax(y_pred_probs[0])

In [ ]:
# convert models prediction probabilities to prediction labels
y_preds = torch.argmax(y_pred_probs, dim=1)

y_preds

In [ ]:
y_blob_test

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

### Training and testing loop

In [ ]:
y_blob_train.dtype

In [ ]:
# Fit the multiclass model to the data
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)

# Set the number of epochs
epochs = 1000

# put the data to the target device
X_blob_train, y_blob_train = X_blob_train.to(device), y_blob_train.to(device)
X_blob_test, y_blob_test = X_blob_test.to(device), y_blob_test.to(device)

for epoch in range(epochs):
    # training
    model_4.train()

    y_logits = model_4(X_blob_train)
    y_pred_probs = torch.softmax(y_logits, dim=1)
    y_preds = torch.argmax(y_pred_probs, dim=1)

    loss = loss_fn(y_logits, y_blob_train)
    acc = accuracy_fn(y_true=y_blob_train, y_pred=y_preds)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # testing
    model_4.eval()
    with torch.inference_mode():
        test_logits = model_4(X_blob_test)
        test_pred_probs = torch.softmax(test_logits, dim=1)
        test_preds = torch.argmax(test_pred_probs, dim=1)

        test_loss = loss_fn(test_logits, y_blob_test)
        test_acc = accuracy_fn(y_true=y_blob_test, y_pred=test_preds)

    if epoch % 100 == 0:
        print(f'Epoch: {epoch} | Loss: {loss:.4f} | accuracy: {acc:.2f}% | Test loss: {test_loss:.4f} | Test accuracy: {test_acc:.2f}%')



In [ ]:
# making and evaluating predictions for multiclass

model_4.eval()
with torch.inference_mode():
    y_logits = model_4(X_blob_test)
    y_pred_probs = torch.softmax(y_logits, dim=1)
    y_preds = torch.argmax(y_pred_probs, dim=1)


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Train')
plot_decision_boundary(model_4, X_blob_train, y_blob_train)

plt.subplot(1, 2, 2)
plt.title('Test')
plot_decision_boundary(model_4, X_blob_test, y_blob_test)

# 13:48:20